<a href="https://colab.research.google.com/github/jmcconne100/P1-Sales-Predictions/blob/main/Jon_McConnell_Project_1_Part_2_(Core).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
filename = "/content/drive/MyDrive/Data_Folders/sales_predictions.csv"
import pandas as pd

In [3]:
sales_DF = pd.read_csv(filename)
sales_DF.info()
# 12 columns and 8,523 rows
# Data types include objects, floats, and ints

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [4]:
sales_DF.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


Data Cleaning

In [7]:
print(sales_DF.duplicated().sum())
sales_DF = sales_DF.drop_duplicates()
# There were none but we went through the motion of removing any potential duplicates

0


In [8]:
sales_DF.isna().sum()
# All Na values are in Item_Weight and Outlet_Size

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [9]:
print(sales_DF['Item_Type'].nunique())
# 16 Item Types so it's reasonable to do a group by

16


In [17]:
filtered_1 = sales_DF.copy()
filtered_1.dropna(subset=['Item_Weight'], inplace=True)
item_Type_Dict = filtered_1.groupby('Item_Type')['Item_Weight'].mean()
print(item_Type_Dict)

Item_Type
Baking Goods             12.277108
Breads                   11.346936
Breakfast                12.768202
Canned                   12.305705
Dairy                    13.426069
Frozen Foods             12.867061
Fruits and Vegetables    13.224769
Hard Drinks              11.400328
Health and Hygiene       13.142314
Household                13.384736
Meat                     12.817344
Others                   13.853285
Seafood                  12.552843
Snack Foods              12.987880
Soft Drinks              11.847460
Starchy Foods            13.690731
Name: Item_Weight, dtype: float64


In [18]:
experimental_Copy = sales_DF.copy()
experimental_Copy['Item_Weight'] = experimental_Copy['Item_Weight'].fillna(experimental_Copy['Item_Type'].map(item_Type_Dict))
experimental_Copy.info()
# I used the average weight by item type from the known weights to create a dictionary that I mapped to the empty values. 
# A little sophisticated given the very small differences in the averages but marginally better than a straight average of all items.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                8523 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 865.6+ KB


In [28]:
sales_DF['Outlet_Identifier'].value_counts()
# So it turns out there are only 10 outlets being analyzed

OUT027    935
OUT013    932
OUT049    930
OUT046    930
OUT035    930
OUT045    929
OUT018    928
OUT017    926
OUT010    555
OUT019    528
Name: Outlet_Identifier, dtype: int64

In [32]:
sales_DF['Outlet_Size'].value_counts()

Medium    2793
Small     2388
High       932
Name: Outlet_Size, dtype: int64

In [38]:
filtered_2 = sales_DF.copy()
filtered_2.dropna(subset=['Outlet_Size'], inplace=True)
filtered_2['Outlet_Size'] = filtered_2['Outlet_Size'].replace('Small',1)
filtered_2['Outlet_Size'] = filtered_2['Outlet_Size'].replace('Medium',2)
filtered_2['Outlet_Size'] = filtered_2['Outlet_Size'].replace('High',3)
Outlet_Dict = filtered_2.groupby(['Outlet_Identifier'])['Outlet_Size'].mean()
print(Outlet_Dict)
# After looking at this it seems there may be some missing outlet sizes for specific outlet identifiers
# After dropping rows with empty outlet sizes we only have 7 outlets so we're left guessing what the remaining 3 sizes are

Outlet_Identifier
OUT013    3.0
OUT018    2.0
OUT019    1.0
OUT027    2.0
OUT035    1.0
OUT046    1.0
OUT049    2.0
Name: Outlet_Size, dtype: float64


In [44]:
Outlet_Type_Dict = filtered_2.groupby(['Outlet_Type'])['Outlet_Size'].median()
print(Outlet_Type_Dict)
# Most grocery stores intuitively have a small footprint and Supermarkets tend to have a median sized footprint
# Notable exception from below was some form of representation of High sized footprints but only one outlet had that size
# It's safe to assume that this will be a useful dictionary map considering that it's not far from 2 or the middle value

Outlet_Type
Grocery Store        1.0
Supermarket Type1    2.0
Supermarket Type2    2.0
Supermarket Type3    2.0
Name: Outlet_Size, dtype: float64


In [45]:
experimental_Copy_2 = sales_DF.copy()
experimental_Copy_2['Outlet_Size'] = experimental_Copy_2['Outlet_Size'].fillna(experimental_Copy_2['Outlet_Type'].map(Outlet_Type_Dict))
experimental_Copy_2.info()
# Now all I need to do is replace the 1,2,3's and merge the experimental copy's into a new copy of the Dataframe

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                8523 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 1.1+ MB


In [46]:
experimental_Copy_2['Outlet_Size'] = experimental_Copy_2['Outlet_Size'].replace(1,'Small')
experimental_Copy_2['Outlet_Size'] = experimental_Copy_2['Outlet_Size'].replace(2,'Medium')
experimental_Copy_2['Outlet_Size'] = experimental_Copy_2['Outlet_Size'].replace(3,'High')

In [47]:
Final_Copy_DF = sales_DF.copy()
Final_Copy_DF['Outlet_Size'] = Final_Copy_DF['Outlet_Size'].fillna(Final_Copy_DF['Outlet_Type'].map(Outlet_Type_Dict))
Final_Copy_DF['Item_Weight'] = Final_Copy_DF['Item_Weight'].fillna(Final_Copy_DF['Item_Type'].map(item_Type_Dict))
Final_Copy_DF.info()
# Looks like we've filled in our data holes successfully

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                8523 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                8523 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 1.1+ MB


In [48]:
Final_Copy_DF['Item_Fat_Content'].value_counts()
# We have a few strange entries in this one

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [51]:
Final_Copy_DF['Item_Type'].value_counts()
# Seemingly no issues in Item Type

Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64

In [52]:
Final_Copy_DF['Item_Fat_Content'] = Final_Copy_DF['Item_Fat_Content'].replace('LF','Low Fat')
Final_Copy_DF['Item_Fat_Content'] = Final_Copy_DF['Item_Fat_Content'].replace('low fat','Low Fat')
Final_Copy_DF['Item_Fat_Content'] = Final_Copy_DF['Item_Fat_Content'].replace('reg','Regular')
Final_Copy_DF['Item_Fat_Content'].value_counts()
# Fixed

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

In [53]:
# Lastly we need some summary statistics for floats and ints columns. Let's grab the info down here to start
Final_Copy_DF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                8523 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                8523 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 1.1+ MB


In [54]:
Final_Copy_DF['Item_Weight'].describe()

count    8523.000000
mean       12.857890
std         4.232804
min         4.555000
25%         9.310000
50%        12.867061
75%        16.000000
max        21.350000
Name: Item_Weight, dtype: float64

In [55]:
Final_Copy_DF['Item_Visibility'].describe()

count    8523.000000
mean        0.066132
std         0.051598
min         0.000000
25%         0.026989
50%         0.053931
75%         0.094585
max         0.328391
Name: Item_Visibility, dtype: float64

In [56]:
Final_Copy_DF['Item_MRP'].describe()

count    8523.000000
mean      140.992782
std        62.275067
min        31.290000
25%        93.826500
50%       143.012800
75%       185.643700
max       266.888400
Name: Item_MRP, dtype: float64

In [57]:
Final_Copy_DF['Outlet_Establishment_Year'].describe()

count    8523.000000
mean     1997.831867
std         8.371760
min      1985.000000
25%      1987.000000
50%      1999.000000
75%      2004.000000
max      2009.000000
Name: Outlet_Establishment_Year, dtype: float64

In [58]:
Final_Copy_DF['Item_Outlet_Sales'].describe()

count     8523.000000
mean      2181.288914
std       1706.499616
min         33.290000
25%        834.247400
50%       1794.331000
75%       3101.296400
max      13086.964800
Name: Item_Outlet_Sales, dtype: float64